In [ ]:
import os
import pandas as pd
import numpy as np

from google.colab import drive

In [ ]:
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

In [ ]:
cd '/content/drive/My Drive/dataset'

In [ ]:
address_df = pd.read_excel('address.xlsx',sheet_name='Sheet1')

In [ ]:
address_df['구']

In [ ]:
bus_information_df = pd.read_excel('bus_information.xlsx',sheet_name='Sheet1')

In [ ]:
bus_stop_df = pd.read_csv('bus_stop.csv',encoding='euc-kr')

In [ ]:
bus_stop_df.drop(['정류장명','주소','위도','경도','Unnamed: 6'], axis=1, inplace=True)

In [ ]:
bus_stop_df

In [ ]:
bus_stop_null = bus_information_df[bus_information_df['주소'].isnull()==True]
                       
bus_stop_null_list = list(bus_stop_null['모바일ID'])         

for i in bus_stop_null_list:
    if type(i) != int:
        error = bus_stop_null_list.index(i)
        bus_stop_null_list.remove(i)
        

bus_stop_null = bus_stop_null[:error]

In [ ]:
bus_stop_null

In [ ]:
bus_stop_list = list(bus_stop_df['관리번호'])

In [ ]:
null_bus_dong=[]
for i in bus_stop_null_list:
    for k in bus_stop_list:
        if i==k:
            null_bus_dong.append(bus_stop_df[bus_stop_df['관리번호']==k]['행정구역'].to_string().split('    ')[1])

In [ ]:
bus_stop_null['행정구역']=null_bus_dong
bus_stop_null

주소 有

In [ ]:
bus_information_full = bus_information_df[bus_information_df['주소'].isnull()==False]
bus_information_full

In [ ]:
bus_information_full_list = list(bus_information_full['주소'])
bus_information_full_dong_list = []
for i in range(len(bus_information_full_list)):
    bus_information_full_dong_list.append(bus_information_full_list[i].split(' ')[1])
    
bus_information_full_dong_list

In [ ]:
bus_information_full_dong_list = bus_information_full_dong_list +list(bus_stop_null['행정구역'])

In [ ]:
address_list=list(address_df['동읍면'])
address_list

In [ ]:
old_dong_list=[]

for i in bus_information_full_dong_list:
    check=False
    for k in address_list:
        if i==k:
            check=True
            break
    if check == False:
        old_dong_list.append(i)

old_dong_set = set(old_dong_list)
old_dong_set

In [ ]:
bus_information_full_dong_list_copy = bus_information_full_dong_list
bus_information_final_list=[]
bus_information_dong_dict={}

for k in old_dong_set:
    
    cnt=0
    for j in bus_information_full_dong_list_copy:
        if j==k:
            cnt+=1
    bus_information_dong_dict[k]=cnt

a=len(old_dong_set)

for i in bus_information_dong_dict:
    for j in range (bus_information_dong_dict[i]):
        bus_information_full_dong_list_copy.remove(i)
    a-=1
    print(i,a)
    new_dong=input('신 주소')
    for k in range (bus_information_dong_dict[i]):
        bus_information_final_list.append(new_dong)

In [ ]:
bus_information_full_dong_list_copy

In [ ]:
bus_information_final = bus_information_full_dong_list_copy+bus_information_final_list

In [ ]:
bus_information_final

In [ ]:
gu_list=[]
dong_list=[]
gu_count_list=[]
dong_count_list=[]

for i in (list(set(address_df['구']))):
    bus_information_gu_count=0
    for j in (list(address_df[address_df['구']==i]['동읍면'])):
        bus_information_dong_count=0
        for k in bus_information_final:
            if j==k:
                bus_information_gu_count+=1
                bus_information_dong_count+=1
                        
        gu_list.append(i)
        dong_list.append(j)
        dong_count_list.append(bus_information_dong_count)

    for k in range (len(list(address_df[address_df['구']==i]['동읍면']))):
        gu_count_list.append(bus_information_gu_count)

In [ ]:
# dong_list
gu_count_list
# dong_count_list

In [ ]:
data = {"구":gu_list,"구별 버스 정보 안내기 수":gu_count_list, "동":dong_list,"동별 버스 정보 안내기 수":dong_count_list}
df = pd.DataFrame(data)
df

In [ ]:
df.to_csv('bus_info_change.csv')